## RPC- Classification

In [ ]:
import splitfolders
import os
import torch,torchvision
from torch.utils.data import DataLoader
from torchvision.transforms import v2




#### Dataset loading and preprocessing

In [2]:
file_name = "dataset"

class_names = os.listdir(file_name)
print(f"class names: {class_names}")

# Splitting folders
splitfolders.ratio(file_name,seed=1337, ratio=(0.8,0.2))

train_path = "./output/train"
test_path = "./output/val"

class names: ['scissors', 'paper', 'rock']


Copying files: 2188 files [00:02, 867.76 files/s]


In [73]:
transform = v2.Compose([
  v2.Resize((64,64)),
  v2.ToTensor(),
  v2.Normalize([0.5,0.5,0.5],[0.5,0.5,0.5])
])

/home/bititude/anaconda3/lib/python3.13/site-packages/torchvision/transforms/v2/_deprecated.py:42: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.Output is equivalent up to float precision.
  warnings.warn(


In [74]:
# preparing data for training

train_dataset = torchvision.datasets.ImageFolder(root=train_path,transform=transform)
test_dataset = torchvision.datasets.ImageFolder(root=test_path,transform=transform)


train_loader = DataLoader(train_dataset,batch_size=32,shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True)

In [75]:
img,label=train_dataset[1234]

In [78]:
img

tensor([[[-0.7412, -0.7333, -0.7412,  ..., -0.7020, -0.7098, -0.7176],
         [-0.7333, -0.7333, -0.7490,  ..., -0.7176, -0.7176, -0.7176],
         [-0.7490, -0.7412, -0.7490,  ..., -0.7333, -0.7412, -0.7412],
         ...,
         [-0.7647, -0.7569, -0.7647,  ..., -0.7412, -0.7412, -0.7412],
         [-0.7647, -0.7490, -0.7647,  ..., -0.7412, -0.7490, -0.7490],
         [-0.7490, -0.7490, -0.7412,  ..., -0.7176, -0.7098, -0.7176]],

        [[-0.0745, -0.0667, -0.0588,  ..., -0.0118, -0.0275, -0.0431],
         [-0.0588, -0.0588, -0.0588,  ..., -0.0039, -0.0196, -0.0275],
         [-0.0431, -0.0353, -0.0353,  ...,  0.0039, -0.0196, -0.0275],
         ...,
         [ 0.0039,  0.0118,  0.0118,  ...,  0.0196,  0.0039,  0.0039],
         [-0.0118,  0.0118,  0.0118,  ...,  0.0118, -0.0039, -0.0118],
         [-0.0275, -0.0118, -0.0039,  ..., -0.0039, -0.0196, -0.0275]],

        [[-0.6784, -0.6784, -0.6784,  ..., -0.6784, -0.6863, -0.7020],
         [-0.6706, -0.6706, -0.6784,  ..., -0